In [2]:
import polars as pl
pl.Config.set_tbl_rows(40)

df = pl.read_csv('temp.csv')
df = df.with_columns(
    [(pl.col(col) * pl.col('S')).round(1).alias(col+'_usd') for col in ['bid_price','ask_price','bid_price_P','ask_price_P']]
)

In [3]:
df.tail(5)

timestamp,bid_price,ask_price,strike,bid_price_P,ask_price_P,expiry,S,tau,F,bid_price_usd,ask_price_usd,bid_price_P_usd,ask_price_P_usd
str,f64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64
"""2025-08-29T12:00:00.000000""",0.013,0.0135,117000,0.063,0.075582,"""19SEP25""",110106.62,0.057078,110618.96,1431.4,1486.4,6936.7,8322.1
"""2025-08-29T12:00:00.000000""",0.011,0.0115,118000,0.063,0.084664,"""19SEP25""",110106.62,0.057078,110618.96,1211.2,1266.2,6936.7,9322.1
"""2025-08-29T12:00:00.000000""",0.008,0.0085,120000,0.063,0.102828,"""19SEP25""",110106.62,0.057078,110618.96,880.9,935.9,6936.7,11322.1
"""2025-08-29T12:00:00.000000""",0.0035,0.0039,125000,0.095,0.148239,"""19SEP25""",110106.62,0.057078,110618.96,385.4,429.4,10460.1,16322.1
"""2025-08-29T12:00:00.000000""",0.0017,0.0021,130000,0.137,0.193649,"""19SEP25""",110106.62,0.057078,110618.96,187.2,231.2,15084.6,21322.1


In [4]:
import numpy as np
from scipy.stats import norm
from scipy.special import ndtr

N = ndtr  # Normal CDF function
# N = norm.cdf

def black76_call(F, K, T, r, vol):
    """
    Black-76 call option price for forward contracts
    
    Parameters:
    F: Forward price
    K: Strike price 
    T: Time to expiration (years)
    r: Risk-free rate (for discounting)
    vol: Volatility
    """
    d1 = (np.log(F/K) + 0.5*vol**2*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return np.exp(-r * T) * (F * N(d1) - K * N(d2))

def black76_put(F, K, T, r, vol):
    """
    Black-76 put option price for forward contracts
    
    Parameters:
    F: Forward price
    K: Strike price
    T: Time to expiration (years) 
    r: Risk-free rate (for discounting)
    vol: Volatility
    """
    d1 = (np.log(F/K) + 0.5*vol**2*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return np.exp(-r * T) * (K * N(-d2) - F * N(-d1))

def black76_vega(F, K, T, r, sigma):
    """
    Black-76 vega (sensitivity to volatility) for forward contracts
    
    Parameters:
    F: Forward price
    K: Strike price
    T: Time to expiration (years)
    r: Risk-free rate  
    sigma: Volatility
    """
    d1 = (np.log(F / K) + 0.5 * sigma ** 2 * T) / (sigma * np.sqrt(T))
    return np.exp(-r * T) * F * norm.pdf(d1) * np.sqrt(T)

# Keep aliases for backward compatibility
bs_call = black76_call
bs_put = black76_put
bs_vega = black76_vega

In [5]:
def find_vol(target_value, F, K, T, r, **kwargs):
    """
    Vectorized implied volatility solver using Newton-Raphson method with Black-76 model.
    
    All arguments can be numpy arrays, scalars, or Polars expressions.
    Returns array of implied volatilities or scalar if all inputs are scalar.
    
    Parameters:
    -----------
    target_value : array-like or scalar
        Option market prices
    F : array-like or scalar  
        Forward prices
    K : array-like or scalar
        Strike prices
    T : array-like or scalar
        Time to expiration (in years)
    r : array-like or scalar
        Risk-free interest rate (for discounting)
    option_type : str, optional
        'C' for call (default), 'P' for put
    
    Returns:
    --------
    numpy.ndarray or float
        Implied volatilities
    """
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    
    # Convert all inputs to numpy arrays for vectorization
    target_value = np.asarray(target_value, dtype=float)
    F = np.asarray(F, dtype=float)
    K = np.asarray(K, dtype=float)
    T = np.asarray(T, dtype=float)
    r = np.asarray(r, dtype=float)
    option_type = kwargs.get('option_type', 'C')
    
    # Get the broadcasted shape
    try:
        shape = np.broadcast(target_value, F, K, T, r).shape
    except ValueError as e:
        raise ValueError(f"Input arrays could not be broadcast together: {e}")
    
    # Initialize volatility array
    sigmas = np.full(shape, 0.5, dtype=float)
    done = np.zeros(shape, dtype=bool)
    
    # Newton-Raphson iterations
    for iteration in range(MAX_ITERATIONS):
        # Calculate option prices and vegas using Black-76
        if option_type == 'P':
            prices = black76_put(F, K, T, r, sigmas)
        else:
            prices = black76_call(F, K, T, r, sigmas)
        
        vegas = black76_vega(F, K, T, r, sigmas)
        
        # Calculate price differences
        diff = target_value - prices
        
        # Check convergence
        converged = np.abs(diff) < PRECISION
        
        # Only update where not converged and vega is not zero
        update_mask = ~done & (vegas != 0) & np.isfinite(vegas) & np.isfinite(diff)
        
        # Newton-Raphson update
        sigmas[update_mask] += diff[update_mask] / vegas[update_mask]
        
        # Ensure volatility stays positive
        sigmas = np.maximum(sigmas, 1e-6)
        
        # Mark converged elements as done
        done = done | converged
        
        # Early exit if all converged
        if np.all(done):
            break
    
    # Return scalar if input was scalar, otherwise return array
    if sigmas.shape == ():
        return float(sigmas)
    else:
        return sigmas

In [6]:
df_vola =\
df.with_columns(
    call_bid_vola = find_vol(
        target_value=df['bid_price_usd'].to_numpy(),
        F=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(),
        r=(interest_rate:=0.08),
        option_type='C'
    ).round(4),
    call_ask_vola = find_vol(
        target_value=df['ask_price_usd'].to_numpy(),
        F=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(), 
        r=interest_rate,
        option_type='C'
    ).round(4),
    put_bid_vola = find_vol(
        target_value=df['bid_price_P_usd'].to_numpy(),
        F=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(),
        r=interest_rate,
        option_type='P'
    ).round(4),
    put_ask_vola = find_vol(
        target_value=df['ask_price_P_usd'].to_numpy(),
        F=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(),
        r=interest_rate,
        option_type='P'
    ).round(4)
)

In [7]:
df_vola

timestamp,bid_price,ask_price,strike,bid_price_P,ask_price_P,expiry,S,tau,F,bid_price_usd,ask_price_usd,bid_price_P_usd,ask_price_P_usd,call_bid_vola,call_ask_vola,put_bid_vola,put_ask_vola
str,f64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2025-08-29T12:00:00.000000""",0.1125,0.156739,95000,0.004,0.0043,"""19SEP25""",110106.62,0.057078,110618.96,12387.0,17258.0,440.4,473.5,0.0,0.7049,0.4642,0.4728
"""2025-08-29T12:00:00.000000""",0.087,0.129493,98000,0.0055,0.0065,"""19SEP25""",110106.62,0.057078,110618.96,9579.3,14258.0,605.6,715.7,0.0,0.6073,0.4265,0.4483
"""2025-08-29T12:00:00.000000""",0.073,0.111328,100000,0.0075,0.0085,"""19SEP25""",110106.62,0.057078,110618.96,8037.8,12258.0,825.8,935.9,0.0,0.542,0.4134,0.4316
"""2025-08-29T12:00:00.000000""",0.073,0.093164,102000,0.0105,0.011,"""19SEP25""",110106.62,0.057078,110618.96,8037.8,10258.0,1156.1,1211.2,0.0,0.476,0.4061,0.4138
"""2025-08-29T12:00:00.000000""",0.073,0.075,104000,0.014,0.0145,"""19SEP25""",110106.62,0.057078,110618.96,8037.8,8258.0,1541.5,1596.5,0.382,0.4089,0.3934,0.4001
"""2025-08-29T12:00:00.000000""",0.066,0.068,105000,0.016,0.0165,"""19SEP25""",110106.62,0.057078,110618.96,7267.0,7487.3,1761.7,1816.8,0.3756,0.4008,0.3857,0.3921
"""2025-08-29T12:00:00.000000""",0.0595,0.0615,106000,0.0185,0.019,"""19SEP25""",110106.62,0.057078,110618.96,6551.3,6771.6,2037.0,2092.0,0.3719,0.3959,0.3811,0.387
"""2025-08-29T12:00:00.000000""",0.053,0.0555,107000,0.021,0.022,"""19SEP25""",110106.62,0.057078,110618.96,5835.7,6110.9,2312.2,2422.3,0.3647,0.3932,0.3729,0.3843
"""2025-08-29T12:00:00.000000""",0.0475,0.049,108000,0.024,0.025,"""19SEP25""",110106.62,0.057078,110618.96,5230.1,5395.2,2642.6,2752.7,0.3651,0.3816,0.367,0.378


In [8]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['call_bid_vola'], mode='markers', name='Call Bid Vola', marker=dict(symbol='triangle-up', color='blue')))
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['call_ask_vola'], mode='markers', name='Call Ask Vola', marker=dict(symbol='triangle-down', color='blue')))
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['put_bid_vola'], mode='markers', name='Put Bid Vola', marker=dict(symbol='triangle-up', color='red')))
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['put_ask_vola'], mode='markers', name='Put Ask Vola', marker=dict(symbol='triangle-down', color='red')))
fig.add_vline(x=df_vola['S'][0], line_color="black", annotation_text="Spot (S)", annotation_position="top")
fig.update_layout(
    title='Implied Volatility vs Strike Price',
    xaxis_title='Strike Price',
    yaxis_title='Implied Volatility',
    yaxis=dict(range=[0.2, 0.8])  # Set the y-axis range as needed
)
fig.show()

In [ ]:
date_str = "20240229"  # Fixed: removed extra digit
df = pl.read_csv('sample_data.csv').with_columns(
            timestamp=(pl.lit(date_str) + " " + pl.col("time")).str.strptime(pl.Datetime('ns'), "%Y%m%d %H:%M:%S%.f"),
        )
df.head()

InvalidOperationError: conversion from `str` to `datetime[ns]` failed in column 'literal' for 1417 out of 1417 values: ["202402029 05:07:30.000000000", "202402029 05:07:30.000000000", … "202402029 05:07:30.000000000"]

You might want to try:
- setting `strict=False` to set values that cannot be converted to `null`
- using `str.strptime`, `str.to_date`, or `str.to_datetime` and providing a format string

In [11]:
df['time'].unique()

time
str
"""03:00:12.000000000"""
"""20:22:42.000000000"""
"""22:35:54.000000000"""
"""09:53:00.000000000"""
"""00:40:18.000000000"""
"""09:36:54.000000000"""
"""03:03:30.000000000"""
"""21:16:24.000000000"""
"""20:11:24.000000000"""
